In [ ]:
import pandas as pd
from pathlib import Path
import json

data_dir = Path('D:\\Documents\\2022\\princeton-hci\\test\\data_inputs')

FIPS = [{"name": "Alabama", "abb": "AL", "code": "01"},
        {"name": "Alaska", "abb": "AK", "code": "02"},
        {"name": "Arizona", "abb": "AZ", "code": "04"},
        {"name": "Arkansas", "abb": "AR", "code": "05"},
        {"name": "California", "abb": "CA", "code": "06"},
        {"name": "Colorado", "abb": "CO", "code": "08"},
        {"name": "Connecticut", "abb": "CT", "code": "09"},
        {"name": "Delaware", "abb": "DE", "code": "10"},
        {"name": "Florida", "abb": "FL", "code": "12"},
        {"name": "Georgia", "abb": "GA", "code": "13"},
        {"name": "Hawaii", "abb": "HI", "code": "15"},
        {"name": "Idaho", "abb": "ID", "code": "16"},
        {"name": "Illinois", "abb": "IL", "code": "17"},
        {"name": "Indiana", "abb": "IN", "code": "18"},
        {"name": "Iowa", "abb": "IA", "code": "19"},
        {"name": "Kansas", "abb": "KS", "code": "20"},
        {"name": "Kentucky", "abb": "KY", "code": "21"},
        {"name": "Louisiana", "abb": "LA", "code": "22"},
        {"name": "Maine", "abb": "ME", "code": "23"},
        {"name": "Maryland", "abb": "MD", "code": "24"},
        {"name": "Massachusetts", "abb": "MA", "code": "25"},
        {"name": "Michigan", "abb": "MI", "code": "26"},
        {"name": "Minnesota", "abb": "MN", "code": "27"},
        {"name": "Mississippi", "abb": "MS", "code": "28"},
        {"name": "Missouri", "abb": "MO", "code": "29"},
        {"name": "Montana", "abb": "MT", "code": "30"},
        {"name": "Nebraska", "abb": "NE", "code": "31"},
        {"name": "Nevada", "abb": "NV", "code": "32"},
        {"name": "New Hampshire", "abb": "NH", "code": "33"},
        {"name": "New Jersey", "abb": "NJ", "code": "34"},
        {"name": "New Mexico", "abb": "NM", "code": "35"},
        {"name": "New York", "abb": "NY", "code": "36"},
        {"name": "North Carolina", "abb": "NC", "code": "37"},
        {"name": "North Dakota", "abb": "ND", "code": "38"},
        {"name": "Ohio", "abb": "OH", "code": "39"},
        {"name": "Oklahoma", "abb": "OK", "code": "40"},
        {"name": "Oregon", "abb": "OR", "code": "41"},
        {"name": "Pennsylvania", "abb": "PA", "code": "42"},
        {"name": "Rhode Island", "abb": "RI", "code": "44"},
        {"name": "South Carolina", "abb": "SC", "code": "45"},
        {"name": "South Dakota", "abb": "SD", "code": "46"},
        {"name": "Tennessee", "abb": "TN", "code": "47"},
        {"name": "Texas", "abb": "TX", "code": "48"},
        {"name": "Utah", "abb": "UT", "code": "49"},
        {"name": "Vermont", "abb": "VT", "code": "50"},
        {"name": "Virginia", "abb": "VA", "code": "51"},
        {"name": "Washington", "abb": "WA", "code": "53"},
        {"name": "West Virginia", "abb": "WV", "code": "54"},
        {"name": "Wisconsin", "abb": "WI", "code": "55"},
        {"name": "Wyoming", "abb": "WY", "code": "56"},
        {"name": "American Samoa", "abb": "AS", "code": "60"},
        {"name": "Guam", "abb": "GU", "code": "66"},
        {"name": "Northern Mariana Islands", "abb": "MP", "code": "69"},
        {"name": "Puerto Rico", "abb": "PR", "code": "72"},
        {"name": "Virgin Islands", "abb": "VI", "code": "78"}]

### Download all state tract info (only needs to be run once)

In [ ]:
import requests
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv('API_KEY')

all_data = [["NAME", "B25001_001E", "state", "county", "tract"]]
for state in FIPS:
    res = requests.get(
        # use 2019 data because 2020 data missing some tracts
        f'https://api.census.gov/data/2019/acs/acs5?get=NAME,B25001_001E&for=tract:*&in=state:{state["code"]}&key={API_KEY}')
    if res.text == '':
        continue
    state_data = res.json()
    all_data.extend(state_data[1:])
    print(state['name'])

with open(data_dir / 'all_housing.json', 'w') as f:
    json.dump(all_data, f)

In [ ]:
zips = pd.read_excel(data_dir / 'zip_lookup.xlsx', index_col=0,
                     dtype={'zip': str, 'tract': str, 'usps_zip_pref_state': str, 'res_ratio': float,
                            'bus_ratio': float, 'oth_ratio': float, 'tot_ratio': float})
zips.rename(columns={'tract': 'geoid'}, inplace=True)

In [ ]:
# # https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html

STATE_LENGTH = 2
COUNTY_LENGTH = 4
TRACT_LENGTH = 6


def split_geoid(geoid):
    return geoid[:STATE_LENGTH], geoid[STATE_LENGTH:STATE_LENGTH + COUNTY_LENGTH - 1], geoid[-TRACT_LENGTH:]

In [ ]:
pdb = pd.read_csv(data_dir / 'pdb.csv', index_col=0, encoding="latin-1", low_memory=False, dtype={286: 'string'})

In [ ]:
housing['geoid']

In [ ]:
import json

with open(data_dir / 'all_housing.json') as f:
    data = json.load(f)

housing = pd.DataFrame(data[1:], columns=data[0])
housing.set_index('tract')


def geoid(state, county, tract):
    return state + county + tract


print(housing[0:1])
housing['geoid'] = housing.apply(lambda x: geoid(x['state'], x['county'], x['tract']), axis=1)

In [ ]:
def get_tracts_in_zip(zip_code):
    res = zips.query('zip == @zip_code')
    return res.to_numpy()


get_tracts_in_zip("98103")

In [ ]:
# GIDTR (11 chars) = State (2 chars) + County (3 chars) + Tract (6 chars)
def get_tract_housing_density(geoid):
    matches = housing.query(f'geoid == @geoid')
    housing_count = int(matches.iloc[0, matches.columns.get_loc('B25001_001E')])
    geoid_int = int(geoid)
    # area is in sq. mi
    land_area = pdb.query('GIDTR == @geoid_int').iloc[0, pdb.columns.get_loc('LAND_AREA')]

    return housing_count / float(land_area), housing_count


get_tract_housing_density("36061003100")

In [ ]:
def get_zip_housing_density(zip_code):
    all_tract_data = get_tracts_in_zip(zip_code)
    weighted_sum = 0
    applied_weights = 0
    for tract in all_tract_data:
        # print(tract[0])
        ratio, weight = get_tract_housing_density(tract[0])
        # print(ratio)
        weighted_sum += ratio * weight
        applied_weights += weight

    return weighted_sum / applied_weights


for z in ["98279", "98103", "98052", "10009"]:
    print(get_zip_housing_density(z))